In [0]:
%scala
val storageAccount = "jalvaradoc"//nombre de la cuenta de almacenamiento
val container = "jalvaradoc" //nombre del contenedor
val sasKey= "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T08:34:00Z&st=2024-12-03T00:34:00Z&spr=https&sig=D%2Fs5tb48A%2FfC4ja8vFUAjUux8Km3LD7IiMPaWdxJYyU%3D"  // llave de conexion

val conf = "fs.azure.sas." + container + "." + storageAccount + ".blob.core.windows.net"

//crear acceso directo
dbutils.fs.mount(source="wasbs://" + container + "@" + storageAccount + ".blob.core.windows.net", mountPoint="/mnt/" + container, extraConfigs=Map(conf -> sasKey))

storageAccount: String = jalvaradoc
container: String = jalvaradoc
sasKey: String = sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T08:34:00Z&st=2024-12-03T00:34:00Z&spr=https&sig=D%2Fs5tb48A%2FfC4ja8vFUAjUux8Km3LD7IiMPaWdxJYyU%3D
conf: String = fs.azure.sas.jalvaradoc.jalvaradoc.blob.core.windows.net
res6: Boolean = true

In [0]:
%fs ls /mnt/jalvaradoc/proyectofinal/

path,name,size,modificationTime
dbfs:/mnt/jalvaradoc/proyectofinal/capabronce/,capabronce/,0,0
dbfs:/mnt/jalvaradoc/proyectofinal/capaoro/,capaoro/,0,0
dbfs:/mnt/jalvaradoc/proyectofinal/capaplata/,capaplata/,0,0
dbfs:/mnt/jalvaradoc/proyectofinal/input/,input/,0,0


In [0]:
#integracion cde archivo Producto.parquet
producto = (spark.read.parquet("/mnt/jalvaradoc/proyectofinal/capaplata/Producto.parquet"))
producto.createOrReplaceTempView("s_producto")

In [0]:
#integracion cde archivo Categoria.parquet
categoria = (spark.read.parquet("/mnt/jalvaradoc/proyectofinal/capaplata/Categoria.parquet"))
categoria.createOrReplaceTempView("s_categoria")

In [0]:
#integracion cde archivo Subcategoria.parquet
subcategoria = (spark.read.parquet("/mnt/jalvaradoc/proyectofinal/capaplata/subcategoria.parquet"))
subcategoria.createOrReplaceTempView("s_subcategoria")

In [0]:
%sql
--Eliminar tabla en caso exista
drop table if exists Fact_Product;

--Crear tabla
create TEMP View Fact_Product
as
select P.Cod_Producto,P.Nombre AS Producto, S.Nombre As SubCategoria, C.Nombre AS Categoria
from s_producto P
join s_subcategoria S on P.Cod_SubCategoria = S.Cod_SubCategoria
join s_categoria C on S.Cod_Categoria = C.Cod_Categoria

In [0]:
dfFactProducto = spark.table("Fact_Product")
dfFactProducto.write.mode("overwrite").parquet("/mnt/jalvaradoc/proyectofinal/capaoro/FactProducto.parquet")

In [0]:
#Eliminar acceso directo
dbutils.fs.unmount("/mnt/jalvaradoc")

/mnt/jalvaradoc has been unmounted.


True